In [8]:
import pandas as pd

df_nodes = pd.read_csv("../Java/resources/nodes_real.csv", sep="\t")
df_nodes.drop(columns=["label", "Id", "timeset"], inplace=True)
df_nodes.rename(columns={"id" : "Spotify ID"}, inplace=True)
df_nodes

,eigencentrality,1,0,2,3,eccentricity,betweenesscentrality,harmonicclosnesscentrality,Spotify ID,degree,triangles,clustering,modularity_class,closnesscentrality,weighted degree
0,0.000000,62.0,Dr. Hook,197908.0,"['bubblegum pop', 'classic rock', 'classic uk ...",0.0,0.0,0.000000,2Mhi3jfuRSdbVZPdjqsnnN,0,0,0.0,0,0.000000,0.0
1,0.000000,0.0,"Johnny Rodriguez, Johnny Russell",7.0,[],0.0,0.0,0.000000,4QFbv1d0BbQANO95hIQaZl,0,0,0.0,1,0.000000,0.0
2,0.002715,35.0,Bill Smith,120.0,[],2.0,0.0,0.833333,4Omy5P9r7PiXYje9h4jMkz,2,1,1.0,3583,0.750000,8.0
3,0.006510,20.0,Cool Breeze,3321.0,[],12.0,0.0,0.180073,5vZr4eKCIxujg3mPZaHGcS,3,3,1.0,4549,0.165762,6.0
4,0.000000,13.0,The Clique,449.0,['freakbeat'],0.0,0.0,0.000000,1TYZpmNJCpmN38laFCVkCm,0,0,0.0,2,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019,0.000000,10.0,Navy Gravy,209.0,[],0.0,0.0,0.000000,5QEZ8qPZSxIvzrkJCxlsn7,0,0,0.0,4748,0.000000,0.0
6020,0.000000,58.0,Sweet Little Band,20088.0,['musica para ninos'],0.0,0.0,0.000000,7HBA3bLuJTLRvjK8NX9ZSy,0,0,0.0,4749,0.000000,0.0
6021,0.000000,38.0,Chico DeBarge,60121.0,"['neo soul', 'new jack swing', 'r&b', 'urban c...",0.0,0.0,0.000000,67ISVBZzcCTTKM17Ps00sx,0,0,0.0,4750,0.000000,0.0
6022,0.002182,NaN,NaN,NaN,NaN,1.0,0.0,1.000000,10aVNpUB6sPjYjunSIIeq5,2,1,1.0,4751,1.000000,4.0


In [9]:
# Define dictionaries for id and title attributes
id_dict = {
    "0": "name",
    "1": "popularity",
    "2": "followers",
    "3": "genres"
}

In [10]:
df_nodes.rename(columns=id_dict, inplace=True)
#df_nodes.sort_values(by="Spotify ID", inplace=True)
df_nodes

,eigencentrality,popularity,name,followers,genres,eccentricity,betweenesscentrality,harmonicclosnesscentrality,Spotify ID,degree,triangles,clustering,modularity_class,closnesscentrality,weighted degree
0,0.000000,62.0,Dr. Hook,197908.0,"['bubblegum pop', 'classic rock', 'classic uk ...",0.0,0.0,0.000000,2Mhi3jfuRSdbVZPdjqsnnN,0,0,0.0,0,0.000000,0.0
1,0.000000,0.0,"Johnny Rodriguez, Johnny Russell",7.0,[],0.0,0.0,0.000000,4QFbv1d0BbQANO95hIQaZl,0,0,0.0,1,0.000000,0.0
2,0.002715,35.0,Bill Smith,120.0,[],2.0,0.0,0.833333,4Omy5P9r7PiXYje9h4jMkz,2,1,1.0,3583,0.750000,8.0
3,0.006510,20.0,Cool Breeze,3321.0,[],12.0,0.0,0.180073,5vZr4eKCIxujg3mPZaHGcS,3,3,1.0,4549,0.165762,6.0
4,0.000000,13.0,The Clique,449.0,['freakbeat'],0.0,0.0,0.000000,1TYZpmNJCpmN38laFCVkCm,0,0,0.0,2,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019,0.000000,10.0,Navy Gravy,209.0,[],0.0,0.0,0.000000,5QEZ8qPZSxIvzrkJCxlsn7,0,0,0.0,4748,0.000000,0.0
6020,0.000000,58.0,Sweet Little Band,20088.0,['musica para ninos'],0.0,0.0,0.000000,7HBA3bLuJTLRvjK8NX9ZSy,0,0,0.0,4749,0.000000,0.0
6021,0.000000,38.0,Chico DeBarge,60121.0,"['neo soul', 'new jack swing', 'r&b', 'urban c...",0.0,0.0,0.000000,67ISVBZzcCTTKM17Ps00sx,0,0,0.0,4750,0.000000,0.0
6022,0.002182,NaN,NaN,NaN,NaN,1.0,0.0,1.000000,10aVNpUB6sPjYjunSIIeq5,2,1,1.0,4751,1.000000,4.0


In [11]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df_nodes is your DataFrame
# Select only the columns to be normalized
columns_to_normalize = df_nodes.columns[5:]  # Exclude the first and last columns
data_to_normalize = df_nodes[columns_to_normalize]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
normalized_data = scaler.fit_transform(data_to_normalize)

# Replace the original data with the normalized values
df_nodes_norm = df_nodes.copy()
df_nodes_norm[columns_to_normalize] = normalized_data

# Print the updated DataFrame
df_nodes_norm


ValueError: could not convert string to float: '2Mhi3jfuRSdbVZPdjqsnnN'

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import numpy as np

# Assuming df is your DataFrame with node attributes and edge attributes added

# Convert the DataFrame to a sparse adjacency matrix
#A_dense = pairwise_distances(df_nodes.drop(columns=['Node']), metric='euclidean')
A_dense = df_nodes_norm.iloc[:,5:]

# Apply Louvain community detection algorithm to detect communities (if needed)
# You may need to find an alternative community detection algorithm for DataFrame-based graphs

# Calculate the modularity for different values of k
k_values = range(1, 11)  # Adjust the range as needed
inertias = []

amount_k = len(k_values)
for k in k_values:
    print(f'{k}/{amount_k}')
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(A_dense)
    inertia = kmeans.inertia_
    inertias.append(inertia)

# Plot the elbow curve
plt.plot(k_values, inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# Assuming df_nodes is your DataFrame containing node attributes

# Extract the features (attributes) from the DataFrame
X = df_nodes_norm.iloc[:,5:]

# Specify the number of clusters (k)
k = 4  # Adjust as needed

# Initialize KMeans model
kmeans = KMeans(n_clusters=k)

# Fit the model to the data
kmeans.fit(X)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Add cluster labels as a new column in the DataFrame
df_nodes['Cluster'] = cluster_labels
df_nodes_norm['Cluster'] = cluster_labels

# Display the DataFrame with cluster labels
df_nodes


In [ ]:
# Assuming df_nodes is your DataFrame with cluster labels

# Calculate the cardinality of each cluster
cluster_cardinality = df_nodes_norm['Cluster'].value_counts()

# Display the cardinality of each cluster
print(cluster_cardinality)
